In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py
import pathlib

PATH = '/storage/home/adz6/group/project'
DIGDATAPATH = os.path.join(PATH, 'datasets', 'data', 'lab', 'dig')
#DAQDATAPATH = os.path.join(PATH, 'daq_data')
#MOTHPATH = os.path.join(PATH, 'moth')
#MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
#PLOTPATH = os.path.join(PATH, 'plots/moth')
#MOTHRESULTS = os.path.join(PATH, 'results/moth')


def SumData(array_data, xpos, ypos, nch, r=0.1, f=25.8e9, antispiral=False):
    
    xx, yy = np.meshgrid(xpos, ypos)
    array_antenna_theta = np.arange(0, 2 * np.pi, 2 * np.pi / array_data.shape[0])

    #xx, yy = np.expand_dims(xx, axis=0).repeat(array_data['angle'].size, axis=0), np.expand_dims(yy, axis=0).repeat(array_data['angle'].size, axis=0)

    xantenna = (r * np.cos(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    yantenna = (r * np.sin(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    
    xdist = xantenna - xx.reshape((1, *xx.shape))
    ydist = yantenna - yy.reshape((1, *yy.shape))
    
    dist_grid = np.sqrt(xdist ** 2 + ydist ** 2)
    
    amplitude_weight = 1 #/ dist_grid
    #amplitude_weight = amplitude_weight * (array_antenna_theta.size / amplitude_weight.sum(axis=0)).reshape((1, amplitude_weight.shape[1], amplitude_weight.shape[2]))

    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    if antispiral:
        
        grid_angles = np.arctan2(ydist, xdist)
        
        antispiral_phase = np.exp(-1j * grid_angles)
        return np.matmul((amplitude_weight * beamforming_phases * antispiral_phase).T, array_data[:, nch, :])
    else:
        return np.matmul((amplitude_weight * beamforming_phases).T, array_data[:, nch, :])



In [ ]:
os.listdir(DIGDATAPATH)

In [ ]:
file = h5py.File(os.path.join(DIGDATAPATH, '211213_cres2_meas_3ch.h5'))

In [ ]:
file['range3cm_step1mm_z0mm'].keys()

In [ ]:
file['range3cm_step1mm_z10mm']['10'].shape

In [ ]:
test_data = file['range3cm_step1mm_z10mm']['2'][:]

In [ ]:
x = np.linspace(-0.05, 0.05, 101)
y = np.linspace(-0.05, 0.05, 101)

select_antennas = np.arange(0, 120, 2)
print(select_antennas, select_antennas.size)

summed_data = SumData(test_data[select_antennas, :, :], x, y, 1, antispiral=False)

In [ ]:
image = np.mean(abs(summed_data) ** 2, axis = -1)

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1)


img = ax.imshow(
    image,
    aspect='auto',
    cmap=cmap,
    extent=(-0.05, 0.05, 0.05, -0.05),
)
cbar = fig.colorbar(img, label=r'$|V_{rms}^2|$')
ax.set_xlabel('X-Position (m)')
ax.set_ylabel('Y-Position (m)')

plt.tight_layout()
save_path = pathlib.Path.home()/'group'/'project'/'plots'/'analysis'/'antenna'/'cyclotron_emitter'
name = '220309_0cm_beamforming_no_antispiral'

#plt.savefig(save_path/name)